In [1]:
import pandas as pd

# reading text-processed data
# df=pd.read_csv("https://raw.githubusercontent.com/Alex-Mak-MCW/SpotifyDataScienceProject/main/Data/NLP_processed_text.csv", encoding='utf-8')

# df=pd.read_csv("https://raw.githubusercontent.com/Alex-Mak-MCW/SpotifyDataScienceProject/main/Data/text_processed_data.csv", encoding='utf-8')

# read test dataset
# df=pd.read_csv("https://raw.githubusercontent.com/Alex-Mak-MCW/SpotifyDataScienceProject/main/Data/PROTO_text.csv", encoding='utf-8')

# reading data directly from textpreprocessing.ipynb
# df=pd.read_csv("https://raw.githubusercontent.com/Alex-Mak-MCW/SpotifyDataScienceProject/main/Sentiment_Analysis/Final_processed_text.csv", encoding='utf-8')

df=pd.read_csv("https://raw.githubusercontent.com/Alex-Mak-MCW/SpotifyDataScienceProject/main/Data/Sentiment_Analysis/test_English.csv", encoding='Utf-8')

In [3]:
# verify dataset
print(df.shape)
df.head()

(1621, 21)


,track_name,artist,album,release_date,duration,popularity,explicit,lyrics,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,mood
0,Blinding Lights,The Weeknd,After Hours,2020-03-20,200040,90,0,yeah tryna call long enough mayb show love may...,0.514,0.730,...,-5.934,1.0,0.0598,0.00146,0.000095,0.0897,0.334,171.005,4.0,0
1,Shape of You,Ed Sheeran,÷ (Deluxe),2017-03-03,233712,86,0,club best place find lover bar go mm friend ta...,0.825,0.652,...,-3.183,0.0,0.0802,0.58100,0.000000,0.0931,0.931,95.977,4.0,1
2,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,2019-05-17,182160,89,0,go time fear one save noth realli got way driv...,0.501,0.405,...,-5.679,1.0,0.0319,0.75100,0.000000,0.1050,0.446,109.891,4.0,0
3,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,Hollywood's Bleeding,2019-09-06,157560,85,0,ayi ayi ayi ayi ooh ooh ooh ooh ooh ooh ayi ay...,0.755,0.522,...,-4.368,1.0,0.0575,0.53300,0.000000,0.0685,0.925,89.960,4.0,1
4,As It Was,Harry Styles,Harry's House,2022-05-20,167303,91,0,come harri want say goodnight holdin back grav...,0.520,0.731,...,-5.338,0.0,0.0557,0.34200,0.001010,0.3110,0.662,173.930,4.0,1


## Technique 1: 
### Count-vectorizer (CV) + Term Frequency-Inverse document Frequency (TF-IDF)

In [4]:
# make copy of df

df_CV_TF_IDF=df.copy()

In [5]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import numpy as np

# Custom transformer to select text column
get_text_data = FunctionTransformer(lambda x: x['lyrics'], validate=False)

# Pipeline for text processing
text_pipeline = Pipeline([
    ('selector', get_text_data),
    ('features', FeatureUnion([
        ('tfidf', Pipeline([
            ('count', CountVectorizer()),
            ('tfidf', TfidfTransformer())
        ]))
    ]))
])

# Get the feature names
text_features = text_pipeline.fit_transform(df_CV_TF_IDF)
feature_names = text_pipeline.named_steps['features'].transformer_list[0][1].named_steps['count'].get_feature_names_out()

# Omit the last 5 feature names
clean_feature_names = feature_names[:-25]

# Create a new CountVectorizer with the filtered feature names
clean_count_vectorizer = CountVectorizer(vocabulary=clean_feature_names)

# Update the pipeline with the new CountVectorizer
text_pipeline.named_steps['features'].transformer_list[0] = ('tfidf', Pipeline([
    ('count', clean_count_vectorizer),
    ('tfidf', TfidfTransformer())
]))

# Transform text data again with the updated pipeline
text_features = text_pipeline.fit_transform(df_CV_TF_IDF)

# print(text_features)

In [6]:
# Get feature names from TF-IDF
tfidf_vectorizer = text_pipeline.named_steps['features'].transformer_list[0][1].named_steps['count']
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

tfidf_feature_names

array(['aa', 'aaaah', 'aaaall', ..., 'zhu', 'zigazig', 'ziggi'],
      dtype=object)

In [7]:
# Access CountVectorizer
count_vectorizer = text_pipeline.named_steps['features'].transformer_list[0][1].named_steps['count']
print(count_vectorizer.get_feature_names_out())

['aa' 'aaaah' 'aaaall' ... 'zhu' 'zigazig' 'ziggi']


In [8]:
# Reset index of both DataFrames
df_CV_TF_IDF.reset_index(drop=True, inplace=True)
# Set feature names as column names for text_features_df
text_features_df = pd.DataFrame(text_features.toarray(), columns=tfidf_feature_names)

# Perform a left merge to align rows properly
df_CV_TF_IDF = pd.merge(df_CV_TF_IDF, text_features_df, left_index=True, right_index=True, how='left', suffixes=('', '_text'))

In [9]:
print(df_CV_TF_IDF.shape)
df_CV_TF_IDF.head()

(1621, 13079)


,track_name,artist,album,release_date,duration,popularity,explicit,lyrics,danceability,energy,...,zeph,zeplin,zeppelin,zero,zerva,zeze,zhou,zhu,zigazig,ziggi
0,Blinding Lights,The Weeknd,After Hours,2020-03-20,200040,90,0,yeah tryna call long enough mayb show love may...,0.514,0.730,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Shape of You,Ed Sheeran,÷ (Deluxe),2017-03-03,233712,86,0,club best place find lover bar go mm friend ta...,0.825,0.652,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,2019-05-17,182160,89,0,go time fear one save noth realli got way driv...,0.501,0.405,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,Hollywood's Bleeding,2019-09-06,157560,85,0,ayi ayi ayi ayi ooh ooh ooh ooh ooh ooh ayi ay...,0.755,0.522,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,As It Was,Harry Styles,Harry's House,2022-05-20,167303,91,0,come harri want say goodnight holdin back grav...,0.520,0.731,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# export feature engineered dataset
df_CV_TF_IDF.to_csv('CV_and_TFIDF.csv', index=False)

## Technique 2: 
### Character Level N-Grams (N=3)

In [11]:
# make df copy

df_N_Grams=df.copy()

In [12]:
print(df_N_Grams.shape)
df_N_Grams.head()

(1621, 21)


,track_name,artist,album,release_date,duration,popularity,explicit,lyrics,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,mood
0,Blinding Lights,The Weeknd,After Hours,2020-03-20,200040,90,0,yeah tryna call long enough mayb show love may...,0.514,0.730,...,-5.934,1.0,0.0598,0.00146,0.000095,0.0897,0.334,171.005,4.0,0
1,Shape of You,Ed Sheeran,÷ (Deluxe),2017-03-03,233712,86,0,club best place find lover bar go mm friend ta...,0.825,0.652,...,-3.183,0.0,0.0802,0.58100,0.000000,0.0931,0.931,95.977,4.0,1
2,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,2019-05-17,182160,89,0,go time fear one save noth realli got way driv...,0.501,0.405,...,-5.679,1.0,0.0319,0.75100,0.000000,0.1050,0.446,109.891,4.0,0
3,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,Hollywood's Bleeding,2019-09-06,157560,85,0,ayi ayi ayi ayi ooh ooh ooh ooh ooh ooh ayi ay...,0.755,0.522,...,-4.368,1.0,0.0575,0.53300,0.000000,0.0685,0.925,89.960,4.0,1
4,As It Was,Harry Styles,Harry's House,2022-05-20,167303,91,0,come harri want say goodnight holdin back grav...,0.520,0.731,...,-5.338,0.0,0.0557,0.34200,0.001010,0.3110,0.662,173.930,4.0,1


In [13]:
# Define a function to generate character-level n-grams
def generate_char_ngrams(text_series, n):
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(n, n))
    # Fit and transform the text data to get the n-grams
    ngrams_matrix = vectorizer.fit_transform(text_series)
    # Get the feature names (n-grams)
    ngrams = vectorizer.get_feature_names_out()
    # Convert the sparse matrix to a dense one and create a DataFrame
    ngrams_df = pd.DataFrame(ngrams_matrix.toarray(), columns=ngrams)
    return ngrams_df

# Generate 3-grams (trigrams)
char_3grams_df = generate_char_ngrams(df_N_Grams['lyrics'], 3)

# print(char_3grams_df)

# Optionally, concatenate the n-grams DataFrame with the original DataFrame
df_with_ngrams = pd.concat([df_N_Grams.reset_index(drop=True), char_3grams_df], axis=1)

In [14]:
print(df_with_ngrams.shape)
df_with_ngrams.head()

(1621, 6835)


,track_name,artist,album,release_date,duration,popularity,explicit,lyrics,danceability,energy,...,zyf,zyn,zz,zza,zzc,zze,zzi,zzl,zzo,zzz
0,Blinding Lights,The Weeknd,After Hours,2020-03-20,200040,90,0,yeah tryna call long enough mayb show love may...,0.514,0.730,...,0,0,0,0,0,0,0,0,0,0
1,Shape of You,Ed Sheeran,÷ (Deluxe),2017-03-03,233712,86,0,club best place find lover bar go mm friend ta...,0.825,0.652,...,0,0,0,0,0,0,0,0,0,0
2,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,2019-05-17,182160,89,0,go time fear one save noth realli got way driv...,0.501,0.405,...,0,0,0,0,0,0,0,0,0,0
3,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,Hollywood's Bleeding,2019-09-06,157560,85,0,ayi ayi ayi ayi ooh ooh ooh ooh ooh ooh ayi ay...,0.755,0.522,...,0,0,0,0,0,0,0,0,0,0
4,As It Was,Harry Styles,Harry's House,2022-05-20,167303,91,0,come harri want say goodnight holdin back grav...,0.520,0.731,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# export feature engineered dataset
df_with_ngrams.to_csv('N_Grams.csv', index=False)

## Technique 3: 
### Word Embeddings

In [17]:
df_Embeddings=df.copy()

#### 3.1 FastText (currently only on google Colab)

In [ ]:
df_FastText=df_Embeddings.copy()

In [ ]:
from gensim.models import FastText

# Assuming df_fast_text is a pandas DataFrame containing the lyrics
# Tokenize text column
df_fast_text['tokenized_lyrics'] = df_fast_text['lyrics'].apply(lambda x: x.split())

# Prepare corpus
corpus = df_fast_text['tokenized_lyrics'].tolist()

# Initialize and train FastText model
fasttext_model = FastText(vector_size=100, window=5, min_count=1)
fasttext_model.build_vocab(corpus_iterable=corpus)
fasttext_model.train(corpus_iterable=corpus, total_examples=len(corpus), epochs=10)

# Example: Get vector for a word
word_vector = fasttext_model.wv['document']

# Example: Get vector for a sentence (average of word vectors)
def get_sentence_vector(sentence):
    words = sentence.split()
    word_vectors = [fasttext_model.wv[word] for word in words if word in fasttext_model.wv]
    return sum(word_vectors) / len(word_vectors) if word_vectors else []

# Assuming there's a text column in df_fast_text
df_fast_text['fasttext_vector'] = df_fast_text['lyrics'].apply(get_sentence_vector)

# Print the resulting vectors
print(df_fast_text['fasttext_vector'])

In [ ]:
df_fast_text.to_csv("FastText.csv", index=False)

#### 3.2 Word2Vec (currently only on google Colab)

In [ ]:
# To be added

#### 3.3 BERT

In [18]:
df_BERT=df.copy()

In [19]:
print(df_BERT.shape)
df_BERT.head()

(1621, 21)


,track_name,artist,album,release_date,duration,popularity,explicit,lyrics,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,mood
0,Blinding Lights,The Weeknd,After Hours,2020-03-20,200040,90,0,yeah tryna call long enough mayb show love may...,0.514,0.730,...,-5.934,1.0,0.0598,0.00146,0.000095,0.0897,0.334,171.005,4.0,0
1,Shape of You,Ed Sheeran,÷ (Deluxe),2017-03-03,233712,86,0,club best place find lover bar go mm friend ta...,0.825,0.652,...,-3.183,0.0,0.0802,0.58100,0.000000,0.0931,0.931,95.977,4.0,1
2,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,2019-05-17,182160,89,0,go time fear one save noth realli got way driv...,0.501,0.405,...,-5.679,1.0,0.0319,0.75100,0.000000,0.1050,0.446,109.891,4.0,0
3,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,Hollywood's Bleeding,2019-09-06,157560,85,0,ayi ayi ayi ayi ooh ooh ooh ooh ooh ooh ayi ay...,0.755,0.522,...,-4.368,1.0,0.0575,0.53300,0.000000,0.0685,0.925,89.960,4.0,1
4,As It Was,Harry Styles,Harry's House,2022-05-20,167303,91,0,come harri want say goodnight holdin back grav...,0.520,0.731,...,-5.338,0.0,0.0557,0.34200,0.001010,0.3110,0.662,173.930,4.0,1


In [20]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Example: Get vector for a sentence using BERT
def get_bert_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

df_BERT['bert_vector'] = df_BERT['lyrics'].apply(get_bert_embedding)
print(df_BERT['bert_vector'])


0       [-0.051606975, 0.059196685, 0.8179265, -0.1613...
1       [-0.25514305, -0.26647133, 0.80487204, -0.1589...
2       [-0.20082025, -0.05428033, 0.6784714, 0.129567...
3       [-0.1918617, 0.11497808, 0.6076808, -0.1166371...
4       [-0.05787892, 0.21525346, 1.042814, -0.1600381...
                              ...                        
1616    [-0.16968827, 0.0020094733, 0.61205995, -0.145...
1617    [0.04698473, 0.037607603, 0.89777786, -0.12191...
1618    [-0.45085692, 0.19648445, 1.0196003, -0.113810...
1619    [-0.3800811, -0.15651153, 0.7316858, -0.078895...
1620    [-0.24994217, 0.067083925, 0.606919, -0.025780...
Name: bert_vector, Length: 1621, dtype: object


In [21]:
# export BERT dataset
df_BERT.to_csv("BERT_Data.csv", index=False)

## Technique 4: 
### Sentiment Intensity Analyzer

In [26]:
df_VADER=df.copy()

In [27]:
print(df_VADER.shape)
df_VADER.head()

(1621, 21)


,track_name,artist,album,release_date,duration,popularity,explicit,lyrics,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,mood
0,Blinding Lights,The Weeknd,After Hours,2020-03-20,200040,90,0,yeah tryna call long enough mayb show love may...,0.514,0.730,...,-5.934,1.0,0.0598,0.00146,0.000095,0.0897,0.334,171.005,4.0,0
1,Shape of You,Ed Sheeran,÷ (Deluxe),2017-03-03,233712,86,0,club best place find lover bar go mm friend ta...,0.825,0.652,...,-3.183,0.0,0.0802,0.58100,0.000000,0.0931,0.931,95.977,4.0,1
2,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,2019-05-17,182160,89,0,go time fear one save noth realli got way driv...,0.501,0.405,...,-5.679,1.0,0.0319,0.75100,0.000000,0.1050,0.446,109.891,4.0,0
3,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,Hollywood's Bleeding,2019-09-06,157560,85,0,ayi ayi ayi ayi ooh ooh ooh ooh ooh ooh ayi ay...,0.755,0.522,...,-4.368,1.0,0.0575,0.53300,0.000000,0.0685,0.925,89.960,4.0,1
4,As It Was,Harry Styles,Harry's House,2022-05-20,167303,91,0,come harri want say goodnight holdin back grav...,0.520,0.731,...,-5.338,0.0,0.0557,0.34200,0.001010,0.3110,0.662,173.930,4.0,1


In [28]:
# VADER

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to apply VADER and return the sentiment scores
def get_vader_sentiment(text):
    scores = analyzer.polarity_scores(text)
    return pd.Series(scores)

# Apply the function to the text column and create new columns for each score
df_VADER[['negative', 'neutral', 'positive', 'compound']] = df_VADER['lyrics'].apply(get_vader_sentiment)

In [29]:
print(df_VADER.shape)
df_VADER.head()

(1621, 25)


,track_name,artist,album,release_date,duration,popularity,explicit,lyrics,danceability,energy,...,instrumentalness,liveness,valence,tempo,time_signature,mood,negative,neutral,positive,compound
0,Blinding Lights,The Weeknd,After Hours,2020-03-20,200040,90,0,yeah tryna call long enough mayb show love may...,0.514,0.730,...,0.000095,0.0897,0.334,171.005,4.0,0,0.141,0.726,0.133,-0.3182
1,Shape of You,Ed Sheeran,÷ (Deluxe),2017-03-03,233712,86,0,club best place find lover bar go mm friend ta...,0.825,0.652,...,0.000000,0.0931,0.931,95.977,4.0,1,0.009,0.574,0.417,0.9996
2,Someone You Loved,Lewis Capaldi,Divinely Uninspired To A Hellish Extent,2019-05-17,182160,89,0,go time fear one save noth realli got way driv...,0.501,0.405,...,0.000000,0.1050,0.446,109.891,4.0,0,0.135,0.580,0.285,0.9852
3,Sunflower - Spider-Man: Into the Spider-Verse,Post Malone,Hollywood's Bleeding,2019-09-06,157560,85,0,ayi ayi ayi ayi ooh ooh ooh ooh ooh ooh ayi ay...,0.755,0.522,...,0.000000,0.0685,0.925,89.960,4.0,1,0.208,0.648,0.144,-0.8979
4,As It Was,Harry Styles,Harry's House,2022-05-20,167303,91,0,come harri want say goodnight holdin back grav...,0.520,0.731,...,0.001010,0.3110,0.662,173.930,4.0,1,0.000,0.753,0.247,0.9538


In [30]:
df_VADER.head().to_csv("VADER.csv", index=False)

## Technique 5: 
### LDA (topic modeling)